In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from gensim.models import KeyedVectors 
filename = r'D:\Desktop_folders\Placement\resume\verification\Sentiment Analysis\GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
from gensim.models import Word2Vec
from keras.datasets import imdb
vocabulary_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded imdb dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))
word_index = imdb.get_word_index()
id2word = {i: word for word, i in word_index.items()}
imdb_dataset = pd.Series(X_train).apply(lambda x: [id2word.get(i) for i in x ]) 
embedding_size=300
word2vec_custom = Word2Vec(imdb_dataset, min_count = 1, size = embedding_size, window = 5)
word2vec_custom.save('word2vec_custom_imdb.bin')

In [ ]:

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
def train_lstm(X_train, y_train, embedding_matrix, embedding_trainable, embedding_size=300, batch_size = 64, num_epochs = 3):
    embedding_size=300 
    model=Sequential()
    model.add(Embedding(vocabulary_size + 1, embedding_size, 
                        weights=[embedding_matrix], input_length=max_words, 
                        trainable = embedding_trainable))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    batch_size = 64
    num_epochs = 3
    from sklearn.model_selection import train_test_split
    X_train2, X_valid, y_train2, y_valid = train_test_split(X_train,y_train, test_size = 0.33, random_state = 123)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

In [ ]:
from keras.datasets import imdb
vocabulary_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

In [ ]:
word_index = imdb.get_word_index()
id2word = {i: word for word, i in word_index.items()}
print("'Review':",[id2word.get(i, ' ') for i in X_train[0]])
print("\n'Label':",y_train[0])

In [ ]:
print('Maximum review length: {}'.format(len(max((X_train + X_test), key=len))))
print('Minimum review length: {}'.format(len(min((X_train + X_test), key=len))))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_words = 1000
X_train = pad_sequences(X_train, maxlen=max_words)
X_test = pad_sequences(X_test, maxlen=max_words)

In [ ]:
embedding_dim = len(word2vec['hi'])
word_index = imdb.get_word_index()
embedding_matrix = np.zeros((vocabulary_size + 1, embedding_dim))
id2word = {i: word for word, i in word_index.items()}
count = 0
for i in range(1,vocabulary_size + 1):
    word = id2word.get(i)
    try: 
        embedding_matrix[i] = word2vec[word]
    except:
        count +=1     
print('No. of words not available in the dictionary:', count)

In [ ]:
train_lstm(X_train, y_train, embedding_matrix = embedding_matrix, embedding_trainable = True)

In [ ]:
train_lstm(X_train, y_train, embedding_matrix = embedding_matrix, embedding_trainable = False)

In [ ]:
import pandas as pd
import numpy as np

from keras.datasets import imdb
vocabulary_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

In [ ]:
word_index = imdb.get_word_index()
id2word = {i: word for word, i in word_index.items()}
word2vec_vocab = list(word2vec.vocab.keys())
X_removed = pd.Series(X_train).apply(lambda x: [id2word.get(i) for i in x if id2word.get(i) in word2vec_vocab])

In [ ]:
word2vec_imdb = Word2Vec.load('word2vec_custom_imdb.bin')
embedding_dim = 300
word_index = imdb.get_word_index()
embedding_matrix = np.zeros((vocabulary_size + 1, embedding_dim))
id2word = {i: word for word, i in word_index.items()}
for i in range(1,vocabulary_size + 1):
    word = id2word.get(i)
    try: 
        embedding_matrix[i] = word2vec[word]
    except:  
        try:
            embedding_matrix[i] = word2vec_imdb[word]
        except:
            print('Why is this word not found in imdb dataset?','Index:',i,', Word:',word)

In [ ]:
train_lstm(X_train, y_train, embedding_matrix = embedding_matrix, embedding_trainable = True)